<a href="https://colab.research.google.com/github/yulin-hou/MyProject/blob/main/chapter-8-resources/boston-dataset-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boston Housing Dataset Example

Most of this code is included in Chapter 8 of [Data Science for Mathematicians](https://ds4m.github.io/).  We convert it to notebook form here so that you can see the output and explore it interactively online yourself.  To run the notebook in a new Google Colab project, click here:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds4m/ds4m.github.io/blob/master/chapter-8-resources/boston-dataset-example.ipynb)

## Step 1: Obtain data

The Boston housing dataset is built into scikit-learn, so we can import it easily, as follows.

In [5]:
import pandas as pd
import numpy as np

def load_boston_data():
    """
    載入波士頓房價數據集。

    Returns:
        tuple: 包含特徵數據 (pandas DataFrame) 和目標變數 (pandas Series) 的元組。
    """
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    target = raw_df.values[1::2, 2]

    features = pd.DataFrame(data, columns=['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'])
    target = pd.Series(target, name='MEDV')  # 將 target 轉換為 pandas Series，並命名為 'MEDV'

    return features, target

# 在 step 1 中呼叫 load_boston_data() 函數
features, target = load_boston_data()

But the `boston` object created this way is a conglomeration of several sub-objects and not ready to be printed in a human-readable way, so we organize it as follows.

## Step 2: Create a feature-target dataset

We extract the features and target into separate variables and inspect their contents.

In [6]:
#import pandas as pd
#features = pd.DataFrame(
#    data=boston.data,
#    columns=boston.feature_names)
#target = boston.target

The features are a pandas DataFrame, the first few rows of which are shown here.

In [7]:
features.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


How many rows are there, actually?

In [8]:
len( features )

506

The target is a NumPy array that can be viewed as another column in the same dataset, as shown here.

In [9]:
target

,MEDV
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2
...,...
501,22.4
502,20.6
503,23.9
504,22.0


## Step 3: Split into training and test datasets

We will use 80% of the data for training and then test our model on the 20% held out for that purpose.  Scikit-learn contains a function that will randomly split the dataset for us into training and test sets.  We add the `random_state` parameter to specify a random number seed, thus guaranteeing reproducibility of the same results if you re-run this notebook later.

In [10]:
from sklearn.model_selection import train_test_split
(X_training, X_test, y_training, y_test) = \
    train_test_split(features,target,train_size=0.8,random_state=1)

Let's verify that the split produced objects of the appropriate sizes.

In [11]:
len( X_training ), len( X_test ), len( y_training ), len( y_test )

(404, 102, 404, 102)

Since 404 is almost exactly 80% of the original 506, it looks like this has worked correctly.

## Step 4: Create a model from the training dataset

We use scikit-learn's Pipeline object to compose two steps in sequence:  First, select the five best features to use for prediction, and second, use those five features to fit a linear model to the training data.

In [12]:
### Step 4: Create a model from the training dataset
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LinearRegression
estimator = Pipeline([
    ('select', SelectKBest(k=5)),
    ('model',  LinearRegression())
])
fit_model = estimator.fit(X=X_training , y=y_training)

Let's say we'd like to see which features were selected.  We can ask the first step in the pipeline to show us its results.

In [13]:
features.columns[ fit_model[0].get_support() ]

Index(['CRIM', 'NOX', 'RM', 'AGE', 'LSTAT'], dtype='object')

And if we want to see the coefficients the model assigned to each of those variables, we can ask teh second step in the pipeline for its results.

In [14]:
fit_model[1].intercept_, fit_model[1].coef_

(np.float64(2.952436726627834),
 array([-0.09549911, -4.08891308,  4.56355544,  0.02161194, -0.61759647]))

The resulting model is therefore approximately the following.
$$ 2.952 - 0.095\text{CRIM} - 4.089\text{NOX} + 4.564\text{RM} + 0.022\text{AGE} - 0.618\text{LSTAT} $$

## Step 5: Score the model using the test set

We compute the root mean squared error of the model on the test set.

In [18]:
# Step 5: 評估模型
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

predictions = fit_model.predict(X=X_test)

# 計算 RMSE
mse = mean_squared_error(y_test, predictions)  # 計算 MSE
rmse = np.sqrt(mse)  # 計算 RMSE

# 計算 MAE
mae = mean_absolute_error(y_test, predictions)

# 計算 MAPE
mape = mean_absolute_percentage_error(y_test, predictions)

# 輸出結果
print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}")

RMSE: 5.63
MAE: 4.52
MAPE: 0.23


random forest

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def train_and_evaluate_rf_regressor(features, target):
    """
    使用整理好的資料，導入 Random Forest Regressor 擬合訓練數據，並列出 RMSE、MAE 和 MAPE。

    Args:
        features: 特徵數據 (pandas DataFrame)。
        target: 目標變數 (pandas Series)。

    Returns:
        None
    """

    # Step 3: 分割數據集
    X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=0.8, random_state=1)

    # 建立 Random Forest Regressor 模型，並設定超參數
    rf_regressor = RandomForestRegressor(
        n_estimators=1500,  # 樹的數量
        max_depth=None,  # 樹的最大深度
        min_samples_split=2,  # 分割節點所需的最小樣本數
        min_samples_leaf=1,  # 葉節點所需的最小樣本數
        max_features='sqrt',  # 每個節點要考慮的特徵數量
        random_state=42  # 隨機種子
    )

    # 使用訓練數據擬合模型
    rf_regressor.fit(X_train, y_train)

    # 使用測試數據進行預測
    predictions = rf_regressor.predict(X_test)

    # 計算 RMSE、MAE 和 MAPE
    mse = mean_squared_error(y_test, predictions)  # 計算 MSE
    rmse = np.sqrt(mse)  # 計算 RMSE
    mae = mean_absolute_error(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)

    # 輸出結果
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}")

# 使用範例
# 假設你已經使用 step 1~3 的程式碼載入了 features 和 target 數據
train_and_evaluate_rf_regressor(features, target)

RMSE: 3.26
MAE: 2.47
MAPE: 0.13


neural network

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

def train_and_evaluate_nn_regressor(features, target, hidden_layer_sizes=(100,)):
    """
    使用整理好的資料，導入神經網路擬合訓練數據，並列出 RMSE、MAE 和 MAPE。

    Args:
        features: 特徵數據 (pandas DataFrame)。
        target: 目標變數 (pandas Series)。
        hidden_layer_sizes: 隱藏層的大小，預設為 (100,)，表示一個包含 100 個神經元的隱藏層。

    Returns:
        None
    """
    # Step 3: 分割數據集
    X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=0.8, random_state=1)

    # 數據標準化
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # 定義超參數網格
    param_grid = {
        'activation': ['relu'],
        'hidden_layer_sizes': [(100, 500)],  # 使用傳入的 hidden_layer_sizes
    }

    # 建立 MLPRegressor 和 GridSearchCV 物件
    nn = MLPRegressor(random_state=42, max_iter=500)  # 設定隨機種子和最大迭代次數
    grid_search = GridSearchCV(estimator=nn, param_grid=param_grid,
                               scoring='neg_mean_squared_error', cv=5)

    # 使用 GridSearchCV 擬合訓練數據
    grid_search.fit(X_train, y_train)

    # 取得最佳模型
    best_nn = grid_search.best_estimator_

    # 使用最佳模型進行預測
    y_pred = best_nn.predict(X_test)

    # 計算 RMSE、MAE 和 MAPE
    mse = mean_squared_error(y_test, y_pred)  # 計算 MSE
    rmse = np.sqrt(mse)  # 計算 RMSE
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # 輸出結果
    print('最佳超參數：', grid_search.best_params_)
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}")
    print(f"MAPE: {mape:.2f}")

# 使用範例
# 假設你已經使用 step 1~3 的程式碼載入了 features 和 target 數據
train_and_evaluate_nn_regressor(features, target, hidden_layer_sizes=(10, 20, 30))

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

最佳超參數： {'activation': 'relu', 'hidden_layer_sizes': (100, 500)}
RMSE: 2.84
MAE: 2.28
MAPE: 0.12
